In [ ]:
import requests,re,time
from lxml import etree
import os,threading
import fnmatch
import shutil

In [ ]:
class sub_spyder():
  def __init__(self,pages,download_path='./zip/'):
    self.domain_name = "https://subscene.com"
    self.cookie = 'LanguageFilter=27; HearingImpaired=2; ForeignOnly=False; _gat=1;'
    self.pages = pages
    self.dp = download_path
    assert self.pages >= 2 and self.pages <= 78
    
    try:
      os.mkdir(self.dp)
    except:
      pass
  
  def get_pages(self):
    pagelist = r"{}/browse/latest/all/".format(self.domain_name)

    for i in range(2,self.pages+1):
      page = "{}{}".format(pagelist,i)
      yield page

  def find_download_page(self):
    headers = {"cookie":self.cookie}
    for page in self.get_pages():
      r = requests.get(page,headers=headers)
      time.sleep(3)
      xmlcontent = etree.HTML(r.text)
      dls = xmlcontent.xpath("//td[@class='a1']/a/@href")
      for dl in dls:
        download = "{}{}".format(self.domain_name,dl)
        # print(download)
        yield download
      print("on page {}\nget {} sites".format(page,len(dls)))

  def get_download_link(self):
    for i in self.find_download_page():
      r = requests.get(i)
      xmlcontent = etree.HTML(r.text)
      downloadlink = xmlcontent.xpath("//div[@class='download']/a/@href")[0]
      downloadlink = "{}{}".format(self.domain_name,downloadlink)
      r_head = requests.head(downloadlink).headers
      if 'Content-Disposition' in r_head:
        content = r_head.get('Content-Disposition')
        filename =  re.findall('(?<=filename=).*',content)[0]

      yield (downloadlink,filename)

  def downloader(self):
    for dl,fn in self.get_download_link():
      r = requests.get(dl,stream=True)
      with open(self.dp+fn,'wb+') as f:
        for chunk in r.iter_content(chunk_size=1024):
          if chunk:
            f.write(chunk)
      time.sleep(1)
      # print("save as {}".format(fn))

In [ ]:
class process_file():
  def __init__(self):
    self.path = "./zip/"
    self.zip_out = "./zip_out/"

    self.mkdir()
    self.unzip()
    self.conv_name()
    self.normalize()
    
  def unzip(self):
    for root,dirs,files in os.walk(self.path):
      for filename in fnmatch.filter(files,"*.zip"):
        cmd = "unzip {} -d {}".format(root+filename,self.zip_out)
        # print(cmd)
        os.system(cmd)
  
  def conv_name(self):
    index = 1
    for root,dirs,files in os.walk(self.zip_out):
      for filename in fnmatch.filter(files,"*.srt"):
        newfile = "{:0>4}.srt".format(index)
        # print(filename,"============",newfile)
        shutil.move(root+filename,self.zip_out+newfile)
        index +=1

  # def remove_charset_err(self):
  #   for root,dirs,files in os.walk(self.zip_out):
  #     for filename in fnmatch.filter(files,"*.srt"):
  #       file_path = root+filename
  #       try:
  #         with open(file_path,'r+',encoding='utf-8') as f:
  #           f.read()
  #           pass
  #       except:
  #         print("charset err {}".format(file_path))
  #         os.remove(file_path)


  def normalize(self):
    for root,dirs,files in os.walk(self.zip_out):
      for filename in fnmatch.filter(files,"*.srt"):
        file_path = root+filename
        # print(file_path)
        try:
          with open(file_path) as fr:
            content = fr.read()
            content = re.sub(r'\d(.*?)\d\n','',content,flags=re.M) #除时间轴
            content = re.sub(r'\d\n','',content,flags=re.M) #除序列
            content = re.sub(r'NETF.*|.*日本語字幕.*','',content,flags=re.M) #除文字信息
            content = re.sub(r'\(.*?\)|（.*?）|\{(.*?)\}','',content,flags=re.M) #除特殊符号
            content = content.replace('\n\n','\n') #除多余换行
            # print(content)

          with open(file_path,"w+") as fw:
            fw.write(content)
            # print(file_path)
        except Exception as E:
          #unicode decode error,need fix
          print(E)
          print('charset err path:%s' % file_path)

  def mkdir(self):
    try:
      os.mkdir(self.zip_out)
    except:
      pass

In [ ]:
def run(pages=70):
  a = sub_spyder(pages)
  start = time.time()
  a.downloader()
  cost = time.time()-start
  print('{:.2f}s'.format(cost))

  start = time.time()
  b = process_file()
  cost = time.time()-start
  print('{:.2f}s'.format(cost))

In [ ]:
run(2)

In [ ]:
!ls ./zip_out